<b>Updated 5th November 2018<br/></b>
Author: Phillip Crout (pc494) <br/> 
Comment: To reflect the move to diffpy

<center><h4> ALERT: This should be downloaded and run, rather than viewed directly on github </h4></center>

In [ ]:
%matplotlib tk
import numpy as np
import hyperspy.api as hs
import pyxem as pxm
import diffpy.structure
from matplotlib import pyplot as plt
from pyxem.generators.indexation_generator import IndexationGenerator
from pyxem.generators.library_generator import VectorLibraryGenerator
from pyxem.generators.indexation_generator import VectorIndexationGenerator
from pyxem.generators.structure_library_generator import StructureLibraryGenerator
from pyxem.utils.sim_utils import peaks_from_best_template
from pyxem.utils.plot import generate_marker_inputs_from_peaks
from pyxem.libraries.structure_library import StructureLibrary

<center><h2>Creating a "fake" dataset</h2></center>

First we set up two crystal structures

In [ ]:
latt = diffpy.structure.lattice.Lattice(5, 5, 5, 90, 90, 90)
atom = diffpy.structure.atom.Atom(atype='Si', xyz=[0, 0, 0], lattice=latt)
si = diffpy.structure.Structure(atoms=[atom], lattice=latt)

In [ ]:
latt = diffpy.structure.lattice.Lattice(3, 3, 5, 90, 90, 120)
atom = diffpy.structure.atom.Atom(atype='Ga', xyz=[0, 0, 0], lattice=latt)
ga = diffpy.structure.Structure(atoms=[atom], lattice=latt)

And some simulation paramaters

In [ ]:
pattern_size = 256  # pixels
half_pattern_size = pattern_size // 2
reciprocal_radius = 1.2
calibration = reciprocal_radius / half_pattern_size
beam_energy = 300.0
ediff = pxm.DiffractionGenerator(beam_energy, 0.025)  # keV and relrod length (1/Å)

We now create 4 seperate patterns, 2 for each crystal, one at 0 degress and one at 10 degrees, but this is all very artificial.

In [ ]:
phase_names = ['Si', 'Ga'] 
sample_lib = StructureLibrary(phase_names, [si, ga], [
                                  [(10,0,0), (0,0,0)],  # For Si
                                  [(0,0,0), (10,0,0)]   # For Ga
                              ])

diff_gen = pxm.DiffractionLibraryGenerator(ediff)
library = diff_gen.get_diffraction_library(sample_lib,
                                           calibration=calibration,
                                           reciprocal_radius=reciprocal_radius,
                                           half_shape=(half_pattern_size, half_pattern_size),
                                           with_direct_beam=False)

data_silicon = []
data_gallium = []

for theta in [0, 10]:
    pattern = library.get_library_entry(
        phase='Si', angle=(theta, 0, 0))['Sim'].as_signal(pattern_size, 0.03, reciprocal_radius)
    data_silicon.append(pattern)
    pattern = library.get_library_entry(
        phase='Ga', angle=(theta, 0, 0))['Sim'].as_signal(pattern_size, 0.03, reciprocal_radius)
    data_gallium.append(pattern)
        
data = [x.data for x in data_silicon] + [x.data for x in data_gallium]

test_data = pxm.ElectronDiffraction(np.asarray(data).reshape(2, 2, pattern_size, pattern_size))
test_data.set_diffraction_calibration(calibration) 

In [ ]:
test_data.plot()

`test_data` now contains the 4 patterns we will attempt to match to, representing our experimental diffraction patterns. We move onto creating a library.

<center><h2> Mapping with template matching </h2></center>

First we need to create a structure library (see the docstrings for details). The structure library generator has an utility function for creating a library covering all unique rotations for the specified structures.

In [ ]:
structure_library_generator = StructureLibraryGenerator(
    [('Si', si, 'cubic'),
     ('Ga', ga, 'hexagonal')])
structure_library = structure_library_generator.get_orientations_from_stereographic_triangle(
    [(0, 10), (0, 10)],  # Inplane rotations to include
    5)  # Angular resolution of the library

The next step generates a library of simulated diffraction data

In [ ]:
diff_gen = pxm.DiffractionLibraryGenerator(ediff)
template_library = diff_gen.get_diffraction_library(structure_library,
                                                    calibration=calibration,
                                                    reciprocal_radius=reciprocal_radius-0.1,
                                                    half_shape=(half_pattern_size, half_pattern_size),
                                                    with_direct_beam=False)

which we then correlate with the test (normally experimental) dataset.

In [ ]:
indexer = IndexationGenerator(test_data, template_library)
match_results = indexer.correlate(n_largest=1, keys=phase_names)

We now have a range of ways of working with this output, but here we simply plot it to show that the method has worked as anticipated.

In [ ]:
match_results.plot_best_matching_results_on_signal(test_data, phase_names, template_library, permanent_markers=False)

Remeber our rotations are anti-clockwise!

<center><h2> Mapping with vector matching </h2></center>

Vector matching starts with finding the diffraction spots using peak finding. The cross correlation `'xc'` method (which works by comparison to an example peak) works good on this example. Using the camera length, we calculate the 3D $\vec g$ that are used for the matching.

In [ ]:
peak_example = test_data.inav[0, 0].isig[122:135, 101:114].data
peaks = test_data.find_peaks('xc', disc_image=peak_example, peak_threshold=0.8)
peaks.calculate_cartesian_coordinates(beam_energy, 0.2)  # Camera length in meters

You can explore other peak finding methods using the interactive viewer.

In [ ]:
#test_data.find_peaks_interactive(peak_example)

Now we create the vector library which contain peak pairs that are matched against the experimental peaks. No rotation list is required for the structure library.

In [ ]:
structure_library = StructureLibrary(['Si', 'Ga'], [si, ga], [[], []])
library_generator = VectorLibraryGenerator(structure_library)
vector_library = library_generator.get_vector_library(reciprocal_radius)

Then, run the indexing. We look for peak pairs with vector lengths within 1.5 pixels and angle within 1° of the experimental peaks.

In [ ]:
indexation_generator = VectorIndexationGenerator(peaks, vector_library)
indexation = indexation_generator.index_vectors(mag_tol=1.5*calibration,
                                                angle_tol=1,
                                                index_error_tol=0.2,
                                                n_peaks_to_index=5,
                                                n_best=2,
                                                keys=['Si', 'Ga'])

From the indexed peaks, we can calculate the crystallographic map for easier access to the indexing results.

In [ ]:
cryst_map = indexation.get_crystallographic_map()

We can, for example, plot the match rate which indicates how successfull the mapping was.

In [ ]:
cryst_map.get_metric_map('match_rate').plot()

Or we can plot the peaks on top of the diffraction patterns.

In [ ]:
indexation.plot_best_matching_results_on_signal(test_data, phase_names,
                                                vector_library, ediff, reciprocal_radius,
                                                permanent_markers=False)

<h2><center> Appendix </center></h2>

Below is playground to help you get used to our rotation convention. "f1" and "f2" will be plotted as in windows with the corresponding name - and each is set to an euler triplet.

In [ ]:
f1 = (90, -3, -90)
f2 = (90, +3, -90)
sample_lib = StructureLibrary(['Si'], [si], [[f1, f2]])
diff_gen = pxm.DiffractionLibraryGenerator(ediff)
library = diff_gen.get_diffraction_library(sample_lib,
                                           calibration=1 / 64,
                                           reciprocal_radius=0.8,
                                           half_shape=(64,64),
                                           with_direct_beam=False)

pattern = library.get_library_entry(phase='Si', angle=f1)['Sim'].as_signal(pattern_size, 0.03, 1)
plt.figure('f1')
plt.imshow(pattern, cmap='viridis', vmax=0.1)

pattern = library.get_library_entry(phase='Si', angle=f2)['Sim'].as_signal(pattern_size, 0.03, 1)
plt.figure('f2')
plt.imshow(pattern, cmap='viridis', vmax=0.1)